# Actividad 3 | Aprendizaje supervisado y no supervisado

Curso: Análisis de grandes volúmenes de datos

Alumno: Luis Daniel Ortega Muñoz | A01795197

In [1]:
import kagglehub
from pyspark.ml.regression import LinearRegression
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StringIndexer, OneHotEncoder, StandardScaler, VectorAssembler
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, hour, date_format, count, round, concat_ws, rand
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

C:\Users\danie\anaconda3\envs\env_pyspark\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Introducción teórica


En este ejercicio se explorarán métodos de aprendizaje de máquina utilizando PySpark. A alto nivel, el aprendizaje de máquina se puede dividir en aprendizaje supervisado y aprendizaje no supervisado.

El aprendizaje supervisado es aquel en el que se tiene un conjunto de datos etiquetado; es decir, para cada elemento en el conjunto sabemos cuál es la "respuesta correcta". En este tipo de técnicas, se busca predecir el resultado de nuevas observaciones a partir de lo aprendido a través del conjunto de datos de entrenamiento. En general, el aprendizaje supervisado se aplica en dos clases de problemas: los problemas de regresión, donde se busca predecir un valor numérico real y continuo, y los problemas de clasificación, en los cuales el objetivo es determinar si un elemento pertenece o no a una categoría. Algunos algoritmos comunes de aprendizaje supervisado son regresión lineal, regresión logística, MLP, DecisionTree, etc.

Por otra parte, en problemas de aprendizaje no supervisado no se cuenta con una variable de salida, por lo que el objetivo general consiste en extraer información y características de nuestro conjunto de datos. Es común utilizar técnicas de aprendizaje no supervisado para realizar clustering (o agrupamiento) de datos, detección de anomalías, para implementar sistemas de recomendación, reducción de dimensionalidad, etc. Algoritmos populares son KMeans, Mean-Shift, PCA, LDA, entre otros.

PySpark proporciona una gran variedad de algoritmos de aprendizaje automático, incluyendo todos los mencionados anteriormente. La lista completa puede ser consultada en la documentación oficial: https://spark.apache.org/docs/latest/api/python/reference/pyspark.ml.html

## Selección de los datos

### Carga del conjunto de datos con PySpark

In [2]:
# Download the latest version
path = kagglehub.dataset_download("sobhanmoosavi/us-accidents")

print("Path to dataset files:", path)

dataset_path = path + "/US_Accidents_March23.csv"

Path to dataset files: C:\Users\danie\.cache\kagglehub\datasets\sobhanmoosavi\us-accidents\versions\13


In [3]:
# Create a Spark session
spark = SparkSession.builder.master("local[*]").getOrCreate()

spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

spark

In [4]:
df = spark.read.csv(dataset_path, header=True, inferSchema=True)

df.show(5)

+---+-------+--------+-------------------+-------------------+-----------------+------------------+-------+-------+------------+--------------------+--------------------+------------+----------+-----+----------+-------+----------+------------+-------------------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+
| ID| Source|Severity|         Start_Time|           End_Time|        Start_Lat|         Start_Lng|End_Lat|End_Lng|Distance(mi)|         Description|              Street|        City|    County|State|   Zipcode|Country|  Timezone|Airport_Code|  Weather_Timestamp|Temperature(F)|Wind_Chill(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Wind_Speed(mph)|Precipitation(in)|Weather_Condition|Ameni

### Sub-muestreo del conjunto de datos

A continuación aplicamos nuestro muestreo del conjunto de datos, tal como se describió en la actividad anterior.

El primer paso consiste en crear las columnas Weather_Condition, Hora_Periodo y Tipo_Día. 

In [5]:
df = df.withColumn("Weather_Type",
    when(col("Weather_Condition").isNull(), "Desconocido")
    .when(col("Weather_Condition").rlike("(?i)null|N/A"), "Desconocido")
    .when(col("Weather_Condition").rlike("(?i)Rain|Drizzle|Thunder|Storm|Snow|Sleet|Hail|Ice|Fog|Haze|Mist|Dust|Sand|Smoke|Wintry|Squall|Tornado|Ash|Funnel"), "Adverso")
    .otherwise("Favorable")
)

df = df.withColumn(
    "Hora_Periodo",
    when(hour("Start_Time") < 6, "Madrugada")
    .when(hour("Start_Time") < 18, "Alta actividad")
    .otherwise("Tarde-Noche")
)

df = df.withColumn("Dia_Semana", date_format("Start_Time", "E"))
df = df.withColumn(
    "Tipo_Día",
    when(col("Dia_Semana").isin("Sat", "Sun"), "Fin de semana").otherwise("Laboral")
)

Posteriormente, filtramos cualquier registro en el cual las columnas clave (Severity, Hora_Periodo, Tipo_Día y Weather_Type) son nulas.

In [6]:
df_filtrado = df.filter(
    (col("Severity").isNotNull()) &
    (col("Hora_Periodo").isNotNull()) &
    (col("Tipo_Día").isNotNull()) &
    (col("Weather_Type").isNotNull())
)

Ahora obtenemos los estratos a partir de las diferentes combinaciones de estas variables, al igual que la probabilidad para cada estrato.

In [7]:
total_registros = df_filtrado.count()

estratos = df_filtrado.groupBy("Severity", "Hora_Periodo", "Tipo_Día", "Weather_Type") \
    .agg(count("*").alias("frecuencia")) \
    .withColumn("probabilidad", round(col("frecuencia") / total_registros, 6)) \
    .orderBy(col("probabilidad").desc())

Para cada estrato, calculamos el número de elementos a incluir a partir del tamaño de la muestra deseado y la probabilidad para cada estrato. En este caso, buscamos una sub-muestra de 10,000 elementos.

In [8]:
# Tamaño total de muestra deseado
n_muestra = 10000

estratos = estratos.withColumn(
    "n_estrato",
    round(col("probabilidad") * n_muestra).cast("integer")
)

Unimos los dataframes con la información de los estratos con nuestro dataset.

In [9]:
# En df_filtrado (base depurada sin nulos en variables clave)
df_filtrado = df_filtrado.withColumn(
    "estrato_id",
    concat_ws("_", "Severity", "Hora_Periodo", "Tipo_Día", "Weather_Type")
)

# Igual en la tabla de estratos con probabilidades y n_estrato
estratos = estratos.withColumn(
    "estrato_id",
    concat_ws("_", "Severity", "Hora_Periodo", "Tipo_Día", "Weather_Type")
)

df_muestreo = df_filtrado.join(
    estratos.select("estrato_id", "n_estrato"),
    on="estrato_id",
    how="inner"
)

Ordenamos de forma aleatoria los elementos dentro de cada estrato.

In [10]:
# Asignar un número aleatorio y calcular el orden por estrato
df_muestreo = df_muestreo.withColumn("rand", rand(seed=42))

window = Window.partitionBy("estrato_id").orderBy("rand")

df_muestreo = df_muestreo.withColumn("row_num", row_number().over(window))

Finalmente, creamos nuestro data frame con la muestra a utilizar, incluyendo sólamente el número de elementos correspondiente a cada estrato.

In [11]:
df_muestra_final = df_muestreo.filter(col("row_num") <= col("n_estrato"))
df_muestra_final.summary()

summary,estrato_id,ID,Source,Severity,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Street,City,County,State,Zipcode,Country,Timezone,Airport_Code,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Weather_Type,Hora_Periodo,Dia_Semana,Tipo_Día,n_estrato,rand,row_num
count,10003,10003,10003,10003,10003,10003,5595,5595,10003,10003,9992,10003,10003,10003,10001,10003,9999,9976,9790,7470,9776,9823,9774,9771,9323,7202,9774,9978,9978,9978,9978,10003,10003,10003,10003,10003,10003,10003
mean,NULL,NULL,NULL,2.212636209137259,36.24469909840036,-94.61993688756752,36.34681220541541,-95.7346700438495,0.5559247225484412,NULL,NULL,NULL,NULL,NULL,56379.03567874911,NULL,NULL,NULL,61.71598569969356,58.311485943775125,64.96389116202946,29.53263870507992,9.099752404338043,NULL,7.593403410919228,0.007060538739239098,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2107.662801159652,6.466151770978525E-4,1054.331400579826
stddev,NULL,NULL,NULL,0.48787772380159755,5.070605427026113,17.38554958180918,5.263693355873905,18.136207202853882,1.597624680398425,NULL,NULL,NULL,NULL,NULL,31097.037297564257,NULL,NULL,NULL,18.9999059851054,22.32139247726831,22.82312340594708,1.0044580229406097,2.5724033627643097,NULL,5.192230141415713,0.041519987983880714,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,1861.2403085753288,3.789342770944724E-4,1234.8936379224372
min,1_Alta actividad_...,A-1000027,Source1,1,24.954442999999998,-124.486977,25.451418,-124.486179,0.0,#1 #2 #3 lane blo...,1/2 Rd,Abbeville,Abbeville,AL,01373-9764,US,US/Central,K04W,-15.0,-31.0,2.0,19.82,0.0,CALM,0.0,0.0,Blowing Snow,Day,Day,Day,Day,Adverso,Alta actividad,Fri,Fin de semana,1,3.88564060815888E-7,1
25%,NULL,NULL,NULL,2,33.427165,-117.217077,33.547271,-117.809275,0.0,NULL,NULL,NULL,NULL,NULL,29334.0,NULL,NULL,NULL,49.0,43.0,49.0,29.36,10.0,NULL,4.6,0.0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,501,3.258897606087663E-4,119
50%,NULL,NULL,NULL,2,35.87199000000001,-87.630844,36.54587,-87.92061,0.034,NULL,NULL,NULL,NULL,NULL,55413.0,NULL,NULL,NULL,64.0,62.0,67.0,29.85,10.0,NULL,7.0,0.0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,995,6.423893092273314E-4,473
75%,NULL,NULL,NULL,2,40.106846000000004,-80.315125,40.25729854,-80.2061,0.489,NULL,NULL,NULL,NULL,NULL,90650.0,NULL,NULL,NULL,76.0,75.0,84.0,30.03,10.0,NULL,10.0,0.0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,4251,9.60305785783433E-4,1751
max,4_Tarde-Noche_Lab...,A-99962,Source3,4,48.930842,-69.907569,48.93099,-69.884072,37.27999877929688,sb 29 jso 11th. 7...,Zorn Ave,Zumbrota,Yuma,WY,99338,US,US/Pacific,KZZV,113.0,111.0,100.0,30.87,60.0,West,33.0,0.99,Wintry Mix,Night,Night,Night,Night,Favorable,Tarde-Noche,Wed,Laboral,4251,0.005325347834556515,4251


## Preparación de los datos

El primer paso es remover las columnas que no son relevantes para nuestros algoritmos de ML. Esto incluye las columnas generadas durante el proceso de sub-muestreo. Por ahora solo nos interesa identificar estas columnas, el proceso de removerlas se realizará posteriormente.

In [12]:
# Comenzamos con la limpieza de las columnas agregadas para el sub muestreo
cols_to_drop = ['ID', 'estrato_id', 'n_estrato', 'rand', 'row_num', 'rand', 'Source']


Ahora eliminamos columnas que no son relevantes para este ejercicio. Esto incluye columnas con información geográfica, ya que para este ejercicio se busca generalizar los hallazgos fuera de Estados Unidos. También eliminamos columnas referentes a la zona horaria y a la hora en que se monitoreó el estado del clima. Aprovechamos también para eliminar columnas redundantes; por ejemplo, las diferentes columnas indicando si es de noche o de día son mayormente redundantes a menos que se busque hacer un análisis muy específico en ese periodo de tiempo. Otro ejemplo es la temperatura del viento, la cual es mayormente redundante en nuestro análisis ya que tenemos la temperatura ambiental registrada.

In [13]:
# Ahora las columnas irrelevantes o redundantes.
cols_to_drop += ['Start_Lng', 'End_Lng', 'Start_Lat', 'End_Lat', 'Street', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone', 'Airport_Code', 'Weather_Timestamp', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight', 'Wind_Chill(F)', 'Description', 'Wind_Direction', 'Sunrise_Sunset']


Ahora manejamos las columnas con valores faltantes. El primer paso es remover cualquier columna donde más del 5% de elementos sea nulo.

In [14]:
# Ignoramos las columnas que ya identificamos como columnas a remover
cols = [c for c in df_muestra_final.columns if c not in cols_to_drop]
total = df_muestra_final.count()

for c in cols:
    n_missing = df_muestra_final.filter(col(c).isNull()).count()
    if n_missing > (total*0.05):
        print(f'Dropping column {c}')
        cols_to_drop.append(c)


Dropping column Wind_Speed(mph)
Dropping column Precipitation(in)


Para el resto del dataset, eliminamos cualquier fila que contenga valores nulos

In [29]:
cols = [c for c in df_muestra_final.columns if c not in cols_to_drop]
df_processed = df_muestra_final.dropna(subset=cols)

Mostramos las columnas a utilizar en nuestro análisis

In [30]:
print(cols)

['Severity', 'Start_Time', 'End_Time', 'Distance(mi)', 'Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Weather_Type', 'Hora_Periodo', 'Dia_Semana', 'Tipo_Día']


Ahora codificamos nuestras variables categóricas mediante el uso de StringIndexer y OneHotEncoder

In [31]:
categorical_columns = ['Weather_Condition', 'Weather_Type', 'Dia_Semana', 'Tipo_Día', 'Hora_Periodo']

# Primero convertimos todas las columnas a índices
for c in categorical_columns:
    indexer = StringIndexer(inputCol=c, outputCol=f"{c}_index", handleInvalid='keep')
    df_processed = indexer.fit(df_processed).transform(df_processed)

df_processed.show(1)

+--------------------+---------+-------+--------+-------------------+-------------------+---------+----------+---------+----------+------------+--------------------+-----------+--------+--------------+-----+----------+-------+----------+------------+-------------------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+------------+------------+----------+-------------+---------+--------------------+-------+-----------------------+------------------+----------------+--------------+------------------+
|          estrato_id|       ID| Source|Severity|         Start_Time|           End_Time|Start_Lat| Start_Lng|  End_Lat|   End_Lng|Distance(mi)|         Description|     Street|    City|        County|State|   Zip

In [32]:
categorical_index_cols = [f"{c}_index" for c in categorical_columns]

encoder = OneHotEncoder(inputCols=categorical_index_cols, outputCols=[f"{c}_vector" for c in categorical_columns], handleInvalid='keep')
df_processed = encoder.fit(df_processed).transform(df_processed)

df_processed.show(1)

+--------------------+---------+-------+--------+-------------------+-------------------+---------+----------+---------+----------+------------+--------------------+-----------+--------+--------------+-----+----------+-------+----------+------------+-------------------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+------------+------------+----------+-------------+---------+--------------------+-------+-----------------------+------------------+----------------+--------------+------------------+------------------------+-------------------+-----------------+---------------+-------------------+
|          estrato_id|       ID| Source|Severity|         Start_Time|           End_Time|Start_Lat| Start_Lng|  En

Ya contamos con la columna `Hora_Periodo` derivada de la hora del accidente, por lo que no nos es tan relevante la hora exacta de los accidentes. Sin embargo, podemos crear una nueva columna derivada con la diferencia entre el momento de inicio y final, expresando los minutos transcurridos hasta que el impacto del accidente terminó.

In [33]:
df_processed = df_processed.withColumn('Minutes', (col('End_Time').cast('long') - col('Start_Time').cast('long')) / 60)

In [34]:
df_processed.show(1)

+--------------------+---------+-------+--------+-------------------+-------------------+---------+----------+---------+----------+------------+--------------------+-----------+--------+--------------+-----+----------+-------+----------+------------+-------------------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+------------+------------+----------+-------------+---------+--------------------+-------+-----------------------+------------------+----------------+--------------+------------------+------------------------+-------------------+-----------------+---------------+-------------------+-------+
|          estrato_id|       ID| Source|Severity|         Start_Time|           End_Time|Start_Lat| Start_

Eliminamos las columnas del DataFrame y persistimos para que PySpark optimice las transformaciones posteriores. Al persistir el DataFrame, nos aseguramos de "congelar" su estado actual, de modo que cualquier operación posterior tenga un punto de partida definido.

In [35]:
for c in cols_to_drop:
    df_processed = df_processed.drop(c)

df_processed = df_processed.persist()

In [36]:
df_processed.summary()

summary,Severity,Distance(mi),Temperature(F),Humidity(%),Pressure(in),Visibility(mi),Weather_Condition,Weather_Type,Hora_Periodo,Dia_Semana,Tipo_Día,Weather_Condition_index,Weather_Type_index,Dia_Semana_index,Tipo_Día_index,Hora_Periodo_index,Minutes
count,9697,9697,9697,9697,9697,9697,9697,9697,9697,9697,9697,9697,9697,9697,9697,9697,9697
mean,2.211096215324327,0.5347730223613786,61.75370733216461,65.04238424254925,29.54614004331239,9.098310817778696,NULL,NULL,NULL,NULL,NULL,3.1973806331855212,0.12962771991337527,2.5004640610498092,0.15839950500154687,0.3834175518201506,613.1597882506622
stddev,0.48565286933328833,1.5032278130919563,18.884921630164534,22.714901433201973,0.9657974153335979,2.6899979781482632,NULL,NULL,NULL,NULL,NULL,5.232705813908283,0.3365242213790287,1.823601002949512,0.3651340174413009,0.6589737826330251,17714.830309716726
min,1,0.0,-15.0,2.0,20.02,0.0,Blowing Dust / Windy,Adverso,Alta actividad,Fri,Fin de semana,0.0,0.0,0.0,0.0,0.0,3.5
25%,2,0.0,49.0,49.0,29.36,10.0,NULL,NULL,NULL,NULL,NULL,0.0,0.0,1.0,0.0,0.0,30.0
50%,2,0.023,64.0,67.0,29.86,10.0,NULL,NULL,NULL,NULL,NULL,2.0,0.0,2.0,0.0,0.0,74.7
75%,2,0.454,76.0,84.0,30.03,10.0,NULL,NULL,NULL,NULL,NULL,4.0,0.0,4.0,0.0,1.0,125.88333333333334
max,4,34.889,114.1,100.0,31.0,70.0,Wintry Mix,Favorable,Tarde-Noche,Wed,Laboral,55.0,2.0,6.0,1.0,2.0,1051256.8166666667


## Preparación del conjunto de entrenamiento y prueba

El primer paso consiste en separar nuestro conjunto de datos en entrenamiento y prueba. Para esto, se utiliza una partición 80/20, que es típica en el dominio del aprendizaje automático.

In [49]:
train_data,test_data = df_processed.randomSplit([0.8,0.2], seed = 42)
print(f"""Existen {train_data.count()} instancias en el conjunto train, y {test_data.count()} en el conjunto test""")

Existen 7799 instancias en el conjunto train, y 1898 en el conjunto test


Ahora procedemos a transformar y preparar nuestro conjunto de datos para entrenamiento. El primer paso es aplicar escalamiento a nuestras variables numéricas. Para este caso, sólo nos enfocaremos en la visibilidad y distancia.

In [50]:
cols_to_scale = ['Distance(mi)', 'Visibility(mi)']

vectorizer = VectorAssembler(inputCols=cols_to_scale, outputCol="numerical_features")
train_data = vectorizer.transform(train_data)
test_data = vectorizer.transform(test_data)

scaler = StandardScaler(inputCol="numerical_features", outputCol="scaled_features")
fitted_scaler = scaler.fit(train_data)
train_data = fitted_scaler.transform(train_data)
test_data = fitted_scaler.transform(test_data)


También removeremos outliers de nuestra variable objetivo. Utilizamos la técnica IQR para esto, calculando los a partir del dataset de entrenamiento.

In [58]:
from pyspark.sql.functions import percentile_approx

# Calculate Q1 and Q3
quantiles = train_data.select(
    percentile_approx('Minutes', [0.25, 0.75], 10000).alias('quantiles')
).collect()[0]['quantiles']

Q1 = quantiles[0]
Q3 = quantiles[1]
IQR = Q3 - Q1

# Filter out outliers
train_data = train_data.filter(
    (col('Minutes') >= Q1 - 1.5 * IQR) &
    (col('Minutes') <= Q3 + 1.5 * IQR)
)
test_data = test_data.filter(
    (col('Minutes') >= Q1 - 1.5 * IQR) &
    (col('Minutes') <= Q3 + 1.5 * IQR)
)

## Construcción de modelos de aprendizaje supervisado y no supervisado

### Aprendizaje supervisado: Regresión lineal

Para este ejemplo, se creará un modelo de regresión lineal que pueda predecir la duración del impacto de un accidente dadas sus características. El primer paso es preparar nuestro vector de características. En este caso, se utilizará Weather_Type, Hora_Periodo, Tipo_Día, Severity y Distancia y Visibilidad. Lo que buscamos es medir el impacto de un accidente según las condiciones en las que sucedió. Debemos usar las columnas previamente procesadas.

In [51]:
cols_to_vectorize = ['Weather_Type_vector', 'Tipo_Día_vector', 'Hora_Periodo_vector', 'Severity', 'scaled_features']
vectorizer = VectorAssembler(inputCols=cols_to_vectorize, outputCol="lr_features")
train_data = vectorizer.transform(train_data)
test_data = vectorizer.transform(test_data)

Ahora creamos nuestro y entrenamos a nuestro modelo de regresión lineal. Indicamos que el vector creado previamente será utilizado como características, mientras que la columna Minutes es nuestra variable objetivo.

In [59]:
lr = LinearRegression(featuresCol = 'lr_features', labelCol = 'Minutes', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_data)


Podemos imprimir los coefficientes de nuestro modelo.

In [60]:
print ("The coefficient of the model is : ", lr_model.coefficients)
print ("The Intercept of the model is : ", lr_model.intercept)

The coefficient of the model is :  [-1.8451326890128377,1.4556126350128107,69.24505988262186,0.0,-4.943759528838038,4.943759528837734,0.0,-5.882001292463004,0.0,7.097347617721259,0.0,-17.9269392886558,7.21616583207795,0.0]
The Intercept of the model is :  121.22484016212401


Asímismo, obtenemos las predicciones con el conjunto de prueba para calcular las métricas de desempeño.

In [61]:
pred_lr = lr_model.evaluate(test_data)

#Root Mean Square Error
eval_lr = RegressionEvaluator(labelCol="Minutes", predictionCol="prediction", metricName="rmse")
rmse_lr = eval_lr.evaluate(pred_lr.predictions)
print("RMSE: %.3f" % rmse_lr)

# Mean Square Error
mse = eval_lr.evaluate(pred_lr.predictions, {eval_lr.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = eval_lr.evaluate(pred_lr.predictions, {eval_lr.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = eval_lr.evaluate(pred_lr.predictions, {eval_lr.metricName: "r2"})
print("r2: %.3f" %r2)

RMSE: 48.611
MSE: 2363.005
MAE: 37.886
r2: 0.078


En este caso, la columna objetivo representa minutos y no fue escalada, por lo que las métricas RMSE y MAE representan minutos también. Por su parte, r2 es un coeficiente, con valores cercanos a 1 indicando un mejor rendimiento para nuestro modelo. MSE representa minutos cuadrados, por lo que para este análisis no nos será de tanta utilidad, sobre todo porque ya contamos con la métrica de RMSE.

Podemos ver que en este caso, las predicciones de nuestro modelo suelen errar por un margen de aproximadamente 35 a 50 minutos, basándonos en las métricas de RMSE y MAE. Si bien esto no es perfecto, coincide con la experiencia personal, ya que los accidentes suelen generar tráfico impredecible y duradero.

Sin embargo, también vemos un valor bajo para r2, indicando que nuestras variables de entrada no explican la varianza en la variable de salida. Esto puede indicar que nuestro modelo se beneficiaría de la apliación de ingeniería de características, de forma que nuestra entrada capture mejor las características de la variable objetivo.

### Aprendizaje no supervisado: K-Means

Para este ejemplo, se utilizará el método de K-Means para obtener agrupamientos en nuestro conjunto de datos. Nuestro objetivo será intentar identificar los diferentes tipos de accidentes que ocurren.

Debido a que este es un problema de aprendizaje no supervisado, no es estrictamente necesario dividir el conjunto de datos en entrenamiento y prueba, puesto que no hay una variable objetivo con la cual validar. Por lo tanto, trabajaremos con el dataset completo.

Para este caso, queremos agrupar accidentes basados en el momento en el que ocurrieron (a partir de las variables Tipo_Dia y Hora_Periodo), las condiciones climáticas (variable Weather_Type), su severidad y las condiciones del camino.

Para esto último, reduciremos todas las variables boolenas a una sola columna que indique si había distracciones.

In [43]:
df_processed = df_processed.withColumn('Distractions',
    (col('Amenity') | col('Bump') | col('Crossing') | col('Give_Way') |
     col('Junction') | col('No_Exit') | col('Railway') | col('Roundabout') |
     col('Station') | col('Stop') | col('Traffic_Calming') |
     col('Traffic_Signal') | col('Turning_Loop')).cast('boolean')
)

df_processed.show(1)

+--------+-------------------+-------------------+------------+--------------+-----------+------------+--------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+------------+--------------+----------+-------------+-----------------------+------------------+----------------+--------------+------------------+------------------------+-------------------+-----------------+---------------+-------------------+-----------------+------------+
|Severity|         Start_Time|           End_Time|Distance(mi)|Temperature(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Weather_Condition|Amenity| Bump|Crossing|Give_Way|Junction|No_Exit|Railway|Roundabout|Station| Stop|Traffic_Calming|Traffic_Signal|Turning_Loop|Weather_Type|  Hora_Periodo|Dia_Semana|     Tipo_Día|Weather_Condition_index|Weather_Type_index|Dia_Semana_index|Tipo_Día_index|Hora_Periodo_index|Weather_Condition_vector|Weather_Type_vector|Dia_Se

Ahora procedemos a vectorizar nuestras características de interés.

In [44]:
cols_to_vectorize = ['Tipo_Día_vector', 'Hora_Periodo_vector', 'Weather_Type_vector', 'Severity', 'Distractions']

vectorizer = VectorAssembler(inputCols=cols_to_vectorize, outputCol="kmeans_features")
kmeans_data = vectorizer.transform(df_processed)

Creamos y entrenamos nuestro modelo de K-Means. En este caso no se indicó un valor para k, esto con la intención de que el modelo lo determine por si mismo.

In [45]:
kmeans = KMeans(featuresCol='kmeans_features')
kmeans_model = kmeans.fit(kmeans_data)

Podemos ahora extraer la información de los agrupamientos obtenidos.

In [46]:
# Printing cluster centers
centers = kmeans_model.clusterCenters()
print(f"Found {len(centers)} clusters")
print("Cluster Centers: ")
for center in centers:
    print(center)

Found 2 clusters
Cluster Centers: 
[1.00000000e+00 0.00000000e+00 0.00000000e+00 7.36061757e-01
 1.74856023e-01 8.90822203e-02 0.00000000e+00 8.72319569e-01
 1.27557897e-01 1.22534003e-04 0.00000000e+00 2.20365151e+00
 3.20794020e-01]
[0.00000000e+00 1.00000000e+00 0.00000000e+00 6.06119792e-01
 2.42838542e-01 1.51041667e-01 0.00000000e+00 8.61328125e-01
 1.38020833e-01 6.51041667e-04 0.00000000e+00 2.25065104e+00
 2.43489583e-01]


En este caso no utilizaremos métricas para evaluar el rendimiento del modelo, sino que intentaremos obtener algún hallazgo mediante la interpretación cualitativa y subjetiva de las agrupaciones obtenidas.

En este caso, podemos observar que se obtuvieron 2 agrupaciones distintas. El centro de cada de ellas nos puede proporcionar hallazgos sobre sus características.

Por ejemplo, considerando que los primeros 2 elementos del vector de características representan el tipo de día (fin de semana o laboral), de primera instancia es evidente que uno de los grupos representan accidentes en fines de semana y el otro representa accidentes en días laborales.

El tercer, séptimo y onceavo elementos del vector representan valores desconocidos para las variables categóricas; el hecho de que en ambos centroides sean cero indica que no tuvieron influencia en la agrupación (y potencialmente no existieron en el dataset).

Las coordenadas de severidad no son muy distintas en ambos centros, así que se puede asumir que no es un factor representativo de estos tipos de accidentes.

Finalmente, podemos ver que los valores del cuarto, quinto y sexto elementos de los ceontroides tienen diferencias significativas. Estos elementos representan la codificación del periodo del día, indicando que los accidentes en fines de semana suelen suceder en horarios distintos a los accidentes durante días laborales.